In [ ]:
import os
import re
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import utils
import keras
import imageio 
from PIL import Image 

In [ ]:
# Reading the data
train = pd.read_csv('agedetectiontrain/train.csv')

# Image resizing of test data into single numpy array
temp = []
for img_name in train.ID:
    img_path = os.path.join('agedetectiontrain/Train', img_name)
    img = imageio.imread(img_path)
    img = np.array(Image.fromarray(img).resize((32, 32))).astype('float32')    
    temp.append(img)

train_x = np.stack(temp)

# Normalizing the images
train_x = train_x / 255.

# Encoding the categorical variable to numeric
lb = LabelEncoder()
train_y = lb.fit_transform(train.Class)
train_y = keras.utils.to_categorical(train_y)

In [ ]:
# Specifying all the parameters we will be using in our network
input_num_units = (32, 32, 3)
hidden_num_units = 500
output_num_units = 3

epochs = 100
batch_size = 128

# Defining the network
model = Sequential([
  InputLayer(input_shape=input_num_units),
  Flatten(),
  Dense(units=hidden_num_units, activation='relu'),
  Dense(units=output_num_units, activation='softmax'),
])

In [ ]:
def models_with_different_optimizers(list_of_optimizers):    
    
    for i in range(len(list_of_optimizers)):        
        model.compile(loss='categorical_crossentropy',
                  optimizer=list_of_optimizers[i], # Learning rate and momentum can be passed inside optimizer
                  metrics=['accuracy'])

        # Traning the model and writing log files for TensorBoard in distinct directories        
        val = re.search('optimizers\..*\so', str(list_of_optimizers[i])).group(0)[11:][:-2] # Fetching optimizer name
        logdir = r'optims\\' + val # Each log file needs to be written in a distinct directory. (Mandatory)
        
        # Writing graph will take time. Hence, keeping it False.
        cb = keras.callbacks.TensorBoard(log_dir=logdir, write_graph=False)         
        print('Building model using '+ val + ' optimizer')
        history = model.fit(train_x, train_y, epochs=epochs, 
                           validation_split=0.2,
                           callbacks=[cb])
        print('Model built sucessfully.')
        print('')

In [10]:
optims = [keras.optimizers.Adam(), keras.optimizers.Adadelta(), 
          keras.optimizers.Adagrad(), keras.optimizers.RMSprop(), 
          keras.optimizers.SGD()]
models_with_different_optimizers(optims)

498/498 ━━━━━━━━━━━━━━━━━━━━ 18s 36ms/step - accuracy: 0.8001 - loss: 0.4742 - val_accuracy: 0.6798 - val_loss: 0.9194
Epoch 15/100
376/498 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.8050 - loss: 0.4685

KeyboardInterrupt: 

#### Run following command in Anaconda prompt after changing directory to the place which holds `optims`
tensorboard --logdir=optims  
In case, if above command doesn't open Tensorboard in the browser then change the port manually as shown:  
tensorboard --logdir=optims/ --host localhost --port 8088  
  
Use .* under tensorboard to visualize all 4 graphs in first section.  
You can also change `Horizontal Axis` from `Step` to `Relative` which describes relative time taken by each algorithm to finish.